# Simple Astra run

In [1]:
from astra import Astra,  AstraGenerator, tools
import os

In [2]:
MY_ASTRA_BIN = os.path.expandvars('$HOME/Code/astra/Astra')
MY_WORKDIR = os.getcwd()+'/work/'
MY_TEMPLATE = os.path.expandvars('$HOME/Code/GitHub/lume-astra/templates/dcgun/astra.in')
tools.mkdir_p(MY_WORKDIR)
os.path.exists(MY_ASTRA_BIN), os.path.exists(MY_TEMPLATE), os.path.exists(MY_WORKDIR)

(True, True, True)

In [3]:
a = Astra(astra_bin=MY_ASTRA_BIN, input_file=MY_TEMPLATE, workdir = 'work')

In [4]:
# Change some inputs
a.input['newrun']['zstop']  = 1
a.input['newrun']['zemit']  = 200
a.input['newrun']['zphase'] = 200
a.input['newrun']['phases'] = True

In [5]:
a.verbose = True
a.run()

init dir:  /Users/chrisonian/Code/GitHub/lume-astra/examples
running astra in /Users/chrisonian/Code/GitHub/lume-astra/templates/dcgun
 --------------------------------------------------------------------------

               Astra- A space charge tracking algorithm 
                            Version 3.0              
                        DESY,  Hamburg 2011          
                        Tue Jul  2 19:02:59 

     Parameter file is:  temp_astra.in                                     
     astra input file for L0 injector (20070501_1)                                   

 Initialize element settings:
     neglecting space charge forces 

 --------------------------------------------------------------------------
 Cavity:

     Reading cavity field data from:      dcgun_GHV.dat                                                                   
     maximum gradient                     -11.60     MV/m
     at                                   3.9750E-02 m
     estimated average g

In [6]:
a.output

{'z': array([0.0050013, 0.010001 , 0.015001 , 0.02     , 0.025001 , 0.030001 ,
        0.034933 , 0.04     , 0.045    , 0.05     , 0.055    , 0.06     ,
        0.065    , 0.07     , 0.075    , 0.08     , 0.085    , 0.09     ,
        0.095    , 0.1      , 0.105    , 0.11     , 0.115    , 0.12     ,
        0.125    , 0.13     , 0.135    , 0.14     , 0.145    , 0.15     ,
        0.155    , 0.16     , 0.165    , 0.17     , 0.175    , 0.18     ,
        0.185    , 0.19     , 0.195    , 0.2      , 0.205    , 0.21     ,
        0.215    , 0.22     , 0.225    , 0.23     , 0.235    , 0.24     ,
        0.245    , 0.25     , 0.255    , 0.26     , 0.265    , 0.27     ,
        0.275    , 0.28     , 0.285    , 0.29     , 0.295    , 0.3      ,
        0.305    , 0.31     , 0.315    , 0.32     , 0.325    , 0.33     ,
        0.335    , 0.34     , 0.345    , 0.35     , 0.355    , 0.36     ,
        0.365    , 0.37     , 0.375    , 0.38     , 0.385    , 0.39     ,
        0.395    , 0.4      , 0.4

In [7]:
a.output.keys()

dict_keys(['z', 't', 'x_average', 'x_rms', 'xp_rms', 'x_normemit', 'xxp_average', 'y_average', 'y_rms', 'yp_rms', 'y_normemit', 'yyp_average', 'E_kinetic', 'z_rms', 'deltaE_rms', 'z_normemit', 'zEp_average', 'run_time', 'start_time'])

In [8]:
a.sim_input_file

'/Users/chrisonian/Code/GitHub/lume-astra/templates/dcgun/temp_astra.in'

In [9]:
a.sim_path

'/Users/chrisonian/Code/GitHub/lume-astra/templates/dcgun'

In [10]:
a.load_screens()

loading 199 screens
[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140.0, 145.0, 150.0, 155.0, 160.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190.0, 195.0, 200.0, 205.0, 210.0, 215.0, 220.0, 225.0, 230.0, 235.0, 240.0, 245.0, 250.0, 255.0, 260.0, 265.0, 270.0, 275.0, 280.0, 285.0, 290.0, 295.0, 300.0, 305.0, 310.0, 315.0, 320.0, 325.0, 330.0, 335.0, 340.0, 345.0, 350.0, 355.0, 360.0, 365.0, 370.0, 375.0, 380.0, 385.0, 390.0, 395.0, 400.0, 405.0, 410.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440.0, 445.0, 450.0, 455.0, 460.0, 465.0, 470.0, 475.0, 480.0, 485.0, 490.0, 495.0, 500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545.0, 550.0, 555.0, 560.0, 565.0, 570.0, 575.0, 580.0, 585.0, 590.0, 595.0, 600.0, 605.0, 610.0, 615.0, 620.0, 625.0, 630.0, 635.0, 640.0, 645.0, 650.0, 655.0, 660.0, 665.0, 670.0, 675.0, 680.0, 685.0, 690.0, 695.0, 700.0, 705.0, 710.0, 715.0

In [11]:
a.screen[0]

{'x': array([ 5.4157e-04, -2.8388e-09,  1.0803e-03, ..., -5.3023e-04,
         1.4246e-04,  7.7722e-04]),
 'y': array([ 4.1989e-09,  5.4385e-04,  1.2893e-08, ..., -1.0852e-03,
        -1.3118e-03, -1.2179e-03]),
 'z_ref': 0.0050065,
 'z_rel': array([ 0.00042169, -0.00040955,  0.00085446, ...,  0.00085192,
        -0.00049069, -0.0013303 ]),
 'px': array([-5.2616e+02, -4.6868e-01, -1.1708e+03, ..., -2.7675e+02,
         7.4828e+02,  1.1090e+02]),
 'py': array([ 7.0478e-01, -4.2050e+02,  1.6372e+00, ...,  1.3667e+03,
         1.0004e+03,  5.0882e+02]),
 'pz_ref': 171900.0,
 'pz_rel': array([  7641.3,  -7714.6,  15205. , ...,  15125. ,  -9436.8, -26196. ]),
 't_ref': 1.0243e-10,
 't_rel': array([ 4.3500e-12, -4.3500e-12,  8.7000e-12, ...,  8.8190e-12,
        -4.5124e-12, -1.4706e-11]),
 'qmacro': array([-1.0001e-14, -1.0001e-14, -1.0001e-14, ..., -1.0001e-14,
        -1.0001e-14, -1.0001e-14]),
 'status': array([3, 3, 3, ..., 5, 5, 5]),
 'species_index': array([1, 1, 1, ..., 1, 1, 1])}

# Write screens to HDF5 in openPMD beamphysics format

In [12]:
from astra import writers
import h5py

In [13]:
H5FILE='test_astra.h5'

In [14]:
h5=h5py.File(H5FILE, 'w')

In [15]:
for i in range(len(a.screen)):
    screen = a.screen[i]
    name = 'screen_'+str(i)
    writers.write_astra_particles_h5(h5, name, screen)   

In [16]:
h5.close()

# Read openPMD

In [17]:
h5 = h5py.File(H5FILE, 'r')

In [18]:
list(h5)[0:10]

['screen_0',
 'screen_1',
 'screen_10',
 'screen_100',
 'screen_101',
 'screen_102',
 'screen_103',
 'screen_104',
 'screen_105',
 'screen_106']

In [19]:
list(h5['screen_0'])[0:10]

['momentum',
 'momentumOffset',
 'particleStatus',
 'position',
 'positionOffset',
 'time',
 'timeOffset',
 'weight']

# Plot openPMD beamphysics

In [20]:
from opmd_beamphysics import bunch_plotting, bunch_tools, bunch_stats

from bokeh.plotting import  show, output_notebook
from bokeh.layouts import column, row
output_notebook(verbose=False, hide_banner=True)

In [23]:
# Pick a screen
bunch1 = h5['screen_198']

# Plot
show(row(
    bunch_plotting.plot_bunch_h5(bunch1, 'x', 'px', bins = 50, liveOnly=False),
    bunch_plotting.plot_bunch_h5(bunch1, 'z', 'pz', bins = 50, liveOnly=False)
))
    
    